In [11]:
import numpy as np
import cv2
import os
import warnings
from tqdm import tqdm
from matplotlib import pyplot as plt
import torch

from sklearn.cluster import KMeans
from sklearn.neighbors import BallTree

from superpoint import SuperPoint

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

database_path = r'./T4imgs/database/'
query_path = r"./T4imgs/queries/"
num_of_imgs = 1

In [14]:
query_path = r"./T4imgs/queries/"

# Extract and Aggregate SIFT Feature of the Database

1. **Extract** and **Aggregate** features from all the database images. I use SIFT here, but SURF or ORB may also work. Every step, we directly read an image from the SSD, extract featrue with help of OpenCV 4.4+, aggregate the extracted feature into a big list `database_entire_des` that would store all the features from all the images, by taking advantage of python's `extend` so we are not creating nested array.
    - It's tempting to store all the loaded images or computed (SIFT) feature and retrieve them later from RAM. But my experiment showed that it's actually faster if we just load them and re-compute (SIFT) feature whenever we need any one of these two, as SSD + CPU doing heavylifting is faster than CPU + RAM doing heavylifing. Anyway, I didn't store the computed SIFT or loaded images anywhere

In [3]:
def describe_SP(img, model):
    model.eval()
    _, inp = read_image(img, device)
    pred = model({'image': inp})
    des = pred['descriptors'][0]
    des = torch.transpose(des, 0, 1)
    des = des.cpu().detach().numpy()
    des = des.astype(np.float64)
    return des

def frame2tensor(frame, device):
    return torch.from_numpy(frame/255.).float()[None, None].to(device)

def read_image(path, device):
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)

    if image is None:
        return None, None, None
    
    inp = frame2tensor(image, device)
    return image, inp

In [4]:
config = {}
model = SuperPoint(config).to(device)

Loaded SuperPoint model


In [5]:
def extract_aggregate_feature(folder_path):
    all_descriptors = list()
    database_feature = dict()
    
    for img_name in tqdm(os.listdir(folder_path)):
        
        # get feature
        img_path = folder_path + img_name
        des = describe_SP(img_path, model)
        
        # append the descriptor to the aggregated list
        all_descriptors.extend(des)
    
    # use np.asarray so we don't copy and save some memory
    all_descriptors = np.asarray(all_descriptors)

    return all_descriptors

In [6]:
database_entire_des = extract_aggregate_feature(database_path)

  0%|          | 0/28600 [00:00<?, ?it/s]f:\Anaconda\envs\dl_class\lib\site-packages\torch\nn\functional.py:4003: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
100%|██████████| 28600/28600 [02:33<00:00, 186.57it/s]


In [7]:
database_entire_des

array([[ 0.07057239, -0.03425601,  0.05497125, ..., -0.08797342,
         0.03104934, -0.03061824],
       [-0.01913537, -0.08570611, -0.01717542, ...,  0.00993874,
        -0.05137511,  0.11027606],
       [-0.04123871, -0.10240069, -0.07178341, ..., -0.03379937,
        -0.04927987,  0.12967685],
       ...,
       [ 0.00099126,  0.02424804, -0.07702713, ..., -0.07886484,
         0.01954492, -0.02025311],
       [ 0.0511143 ,  0.07189209, -0.0496801 , ..., -0.0844245 ,
         0.02476264, -0.00896518],
       [-0.01272065, -0.08381266,  0.04697344, ...,  0.09732965,
        -0.03126185, -0.01892946]])

# Clustering the Features using K-mean

2. **Clustering** the entire list of features by using scikit-learn's KMeans class. 
    - We cluster them into 16 class and only run it 1 times, because in our case we are actually finding the identical image in the database given a query, and the image is very small itself. In the real world the query image may be larger and not even in the database, so we can only find similar one, we would need more classes (64 or 256) and more run of the clustering algorithm to get the best performing one (like 10 times) so the generated codebook is more robust.
    - I turned off the text display for training so it's less verbose.

In [8]:
# clustering the entire bag of descriptor
codebook = KMeans(n_clusters = 16, init='k-means++', n_init=1, verbose=1).fit(database_entire_des)

Initialization complete
Iteration 0, inertia 1238436.8184479217
Iteration 1, inertia 805091.2034954784
Iteration 2, inertia 790993.4822269739
Iteration 3, inertia 785865.9797691304
Iteration 4, inertia 783332.1120288162
Iteration 5, inertia 781552.5783317005
Iteration 6, inertia 779885.8957516701
Iteration 7, inertia 778215.1441894841
Iteration 8, inertia 776725.6306437213
Iteration 9, inertia 775546.6877727702
Iteration 10, inertia 774690.0073707501
Iteration 11, inertia 774061.6008872973
Iteration 12, inertia 773565.5086621407
Iteration 13, inertia 773157.7935120404
Iteration 14, inertia 772824.1843550209
Iteration 15, inertia 772557.6341004377
Iteration 16, inertia 772343.3817622741
Iteration 17, inertia 772169.794354088
Iteration 18, inertia 772020.4949608071
Iteration 19, inertia 771888.6392220616
Iteration 20, inertia 771771.2259632677
Iteration 21, inertia 771665.3044280304
Iteration 22, inertia 771568.6293931158
Iteration 23, inertia 771479.6390612455
Iteration 24, inertia 7713

# Compute VLAD Feature for Every Database Image

3. **Compute** VLAD feature for every image. We first load all images, compute their (SIFT) features on the fly as it's faster like I said before. We then use this feature to compute the corresponding VLAD feature of this image, and append it to the list `database_VLAD`, where each element is a VLAD feature.
    - We compute VLAD by compute the sum of residue to each centroid and concatenate these vectors.
    - We normalize the VLAD vector using square root normalization and L2 normalization.
    - My implementation of VLAD calculation is adapted from here: https://github.com/jorjasso/VLAD/blob/eeaad96c33aea9c11bceb285ba5cdabba9fb663f/VLADlib/VLAD.py#L177
4. At the same time we create a list `database_name` used to hold all the names of database image. Because we are inserting name to this list at the same time we create and append a VLAD feature, we now have a one-to-one mapping between `database_VLAD` and `database_name`, i.e. two list is pointing to the same image if given two identical index.

In [9]:
def get_VLAD(X, codebook):

    predictedLabels = codebook.predict(X)
    centroids = codebook.cluster_centers_
    labels = codebook.labels_
    k = codebook.n_clusters
   
    m,d = X.shape
    VLAD_feature = np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels == i) > 0:
            # add the diferences
            VLAD_feature[i] = np.sum(X[predictedLabels==i,:] - centroids[i],axis=0)
    

    VLAD_feature = VLAD_feature.flatten()
    # power normalization, also called square-rooting normalization
    VLAD_feature = np.sign(VLAD_feature)*np.sqrt(np.abs(VLAD_feature))

    # L2 normalization
    VLAD_feature = VLAD_feature/np.linalg.norm(VLAD_feature)
    return VLAD_feature

In [12]:
warnings.filterwarnings('ignore')

database_VLAD = list()
database_name = list()
for img_name in tqdm(os.listdir(database_path)):
    img_path = database_path + img_name
    des = describe_SP(img_path, model)
    VLAD = get_VLAD(des, codebook)
    database_VLAD.append(VLAD)
    database_name.append(img_name)
    
database_VLAD = np.asarray(database_VLAD)
    #database_VLAD[VLAD] = img_name

100%|██████████| 28600/28600 [09:33<00:00, 49.86it/s]


# Indexing all the VLAD Features

5. **Indexing** all the VLAD features by creating a `BallTree` of the list `database_VLAD`. This is not essential because we can also do a pair-wise comparison, but BallTree saves a lot of time when retrieving the item that has the smallest distance to the query. This is not essential so I will skip explaining Balltree. But generally, it's a efficient indexing method that performs better when the data is high dimensional, comparing to its alternative KD-Tree
    - I am using L2 distance as the measure between VLAD features. But the choice doesn't matter in this specific problem, because again, we are finding the exact same picture so the distance, no matter what, would be 0, since SIFT and VLAD are both not randomized, so for the same image they would generate the same feature vector.

In [13]:
tree = BallTree(database_VLAD, leaf_size=60)

# Query

6. Compute (SIFT) feature of all the query images, and then compute VLAD feature accordingly, using the same clustering as the database. We then get the VLAD feature of this 5 query images. We then find the images in database whose VLAD feature distance to these query images are 0, respectively.
    - Here we are finding only the top 1 match in `tree.query`, because we know the distance will be 0 as the image in query will have an identical one in the database. In the real world when finding similar images, we will need more matching like 3 or 5, and manually or use some other algorithm to further identify the most similar one

In [15]:
value_list = list()
for img_name in tqdm(os.listdir(query_path)):
    image_path = query_path+img_name
    q_des = describe_SP(image_path, model)
    query_VLAD = get_VLAD(q_des, codebook).reshape(1, -1)
    
    # we only want the cloest one
    dist, index = tree.query(query_VLAD, num_of_imgs)
    
    # index is an array of array of 1
    value_name = database_name[index[0][0]]
    value_list.append(value_name)

100%|██████████| 5/5 [00:00<00:00,  5.75it/s]


# Results

7. Below is the matches in the database for all the query images, in the order from query1.png to query5.png

In [16]:
value_list

['image1373.png',
 'image2622.png',
 'image6051.png',
 'image26588.png',
 'image13935.png']